In [1]:
import pandas as pd
import random
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)
len(df)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


72947

In [5]:
df = df.dropna()
len(df)

70208

In [6]:
toxic_fields = [
    'toxic_score',
    'toxic_score.1',   
    'toxic_score.2',
    'toxic_score.3',    
    'toxic_score.4',    
]

In [7]:
def get_avg_toxicity(row):
    result = sum(row[field] for field in toxic_fields)/20.0
    for field in toxic_fields:
        diff = abs(row[field]/4.0 - result)
        if (diff > 0.2):
            return None
    return result

In [8]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [9]:
df = df.dropna()
len(df)

27994

In [10]:
def get_bucket(row, field):
    if (row[field] < 0.6):
        return 0.0
    elif (row[field] < 0.7):
        return 0.6
    elif (row[field] < 0.8):
        return 0.7
    elif (row[field] < 0.9):
        return 0.8
    else:
        return 0.9

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)

In [12]:
myrange = [0.6, 0.7, 0.8, 0.9]
sampled_dfs = []
for ow in myrange:
    g = df[df['avg_toxic_window'] == ow]
    print (ow, len(g))
    sampled_dfs.append(g.sample(4))

0.6 111
0.7 30
0.8 61
0.9 82


In [13]:
final_df = pd.concat(sampled_dfs)

In [14]:
def get_random_date():
    random_number_of_days = random.randrange(40)
    random_date = datetime.date(2021, 10, 1) + datetime.timedelta(days=random_number_of_days)
    return random_date

In [15]:
def assignPeripherals(final_df):
    final_df = final_df[['comment', 'avg_toxic_score', 'comment_ID']]
    
    # Set username
    usernameDf = pd.read_csv('usernames.csv')
    usernames = set(list(usernameDf['username']))
    final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
    
    # Set full name
    nameDf = pd.read_csv('random_names.csv')
    names = set(list(nameDf['name']))
    final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)    
    
    # Set integer fields
    final_df['num_likes'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_comments'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_retweets'] = np.random.randint(1, 100, final_df.shape[0])        
    
    # Set publication date
    final_df['pub_date'] = df.apply(lambda row: get_random_date(), axis=1)
    
    return final_df

In [16]:
final_df = assignPeripherals(final_df)
final_df.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
8496,This is why we really can’t protect our youth from society’s view of black teens... unsupervised and out of control...and just utterly ridiculous behavior 😲,0.6,852_13,NoteworthyMuskrat,Victoria Scott,25,35,30,2021-10-09
53139,"Im sitting here spooning honey into my mouth like the fat whore you’re pretending you’re not. Anyways, when I’m 35, I’m pretty sure I’ll be sexy too! Just like you guys!",0.6,4563_15,bravelypracticalchina,Preston Owens,20,41,90,2021-10-15
24583,"Purchases used jets from Australia...takes soldiers in combat areas danger pay from them but lets fucking work to get that UN seat since Ireland and Norway pulled ahead, by continuing this Climate nonsense! Our military is so under equipped and in need of training but climate! 🖕",0.6,3527_8,nostalgic_narrator,Connie Barnes,77,41,61,2021-10-12
3607,"#Alright News California imam prays for the extermination of Jews, other Muslim leaders respond “Ameen!”",0.6,1182_13,therapeutic_carrier,Tess Crawford,80,3,29,2021-10-24
17261,"You are out of your league and out of your mind. If you don’t think you will made to pay, you are even dumber than you appear, which astounds the mind. GFY.",0.7,1150_8,neglected_unhappiness,Lydia Bennett,40,74,4,2021-10-03


In [17]:
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.to_csv('main_comments.csv', index=False)

In [18]:
nontoxic_df = df.query('avg_toxic_window == 0.0')

In [19]:
nontoxic_sample = nontoxic_df.sample(20)

In [20]:
nontoxic_sample = assignPeripherals(nontoxic_sample)
nontoxic_sample.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
27939,What’s paining me if you people are yabbing broke women Cos I’m broke and I can do stuff for the man I love,0.05,2399_16,favorableheads3963,Dale Thomas,41,12,98,2021-10-08
23904,"I'm just shocked the MAGA crowd isn't with us since they were pretty amped up about all that Pizza Gate BS. I'd think they'd be really excited about fighting the pedophiles and traffickers but ya know, for real this time.",0.40,3218_13,thefrizzyforgiveness,Lucia Cunningham,42,28,50,2021-10-30
22801,"Crossover/OC friendly: *Moderately descriptive *DM/Tag to roleplay if you wish *TOS, Star Trek: Disco, AOS, TNG, DS9, Voyager, Enterprise, Star Wars, Orville RP, Mass Effect/Destiny, Sci-fi rp in general",0.00,2074_10,thelyricalmirth,Alberta Cameron,64,76,41,2021-10-25
6732,Not more than ass we don't,0.05,1473_18,orderly_reviewer,Eric Bennett,16,8,97,2021-10-03
26846,Joanna is SOOO BORINGG!! BOOOOOOOOO🍅🍅🍅🍅🍅🍅🍅🍅🍅,0.05,3491_12,LastVictory,Vincent Craig,70,37,90,2021-11-09


In [21]:
nontoxic_sample.to_csv('nontoxic_comments.csv', index=False)